In [4]:
!pip3 install implicit --upgrade

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import implicit
import scipy.sparse as sparse

In [6]:
articles = 'articles.xlsx'
customers = 'customers.xlsx'
transaction_val = 'transactions_val.xlsx'
users_test = 'users_test.xlsx'
transactions_train = 'transactions_train.csv'

In [7]:
df_articles = pd.read_excel(articles)
df_customers = pd.read_excel(customers)
df_transaction_val = pd.read_excel(transaction_val)
df_users_test = pd.read_excel(users_test)
df_transaction_train = pd.read_csv(transactions_train)

In [8]:
# Definicion de métricas (No editar)
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0
    return dcg_at_k(r, k) / idcg

In [9]:
df_transaction_val

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id
0,0,2020-09-20,1f293f035403ce2d1ff0cdb23cc3faf8b1c2ad04dbedbe...,535035001,0.118627,2
1,1,2020-06-15,b4ba419fd5afc5eb274ab3ae1638a51a2e916b4deb0a5c...,654046004,0.022017,2
2,2,2019-10-20,1a1563c28f0d8781ad87f76e0a5af0485003e350f541dd...,742086008,0.033881,2
3,3,2020-07-14,3219f007e898957cbe0ff0f22010cd7118c06f7498a167...,778534006,0.042356,1
4,4,2020-09-07,4e5c03f7ffefe2f99f8e0c9d9805c1571867e6a8dd7091...,706016015,0.033881,1
...,...,...,...,...,...,...
140307,140307,2019-09-22,3c0c198b6c04d50b639ca8128c3010603f34b37896bbf8...,768205001,0.025407,2
140308,140308,2019-12-18,3c0c198b6c04d50b639ca8128c3010603f34b37896bbf8...,756040001,0.006763,1
140309,140309,2019-12-18,3c0c198b6c04d50b639ca8128c3010603f34b37896bbf8...,731851001,0.008458,1
140310,140310,2020-06-15,3c0c198b6c04d50b639ca8128c3010603f34b37896bbf8...,858678001,0.059305,2


In [10]:
df_transaction_val.nunique()

Unnamed: 0          140312
t_dat                  734
customer_id          42513
article_id           30252
price                 3050
sales_channel_id         2
dtype: int64

In [11]:
user_items_test = {}

for row in df_transaction_val.itertuples():
    if row[3] not in user_items_test:
        user_items_test[row[3]] = []

    user_items_test[row[3]].append(row[4])

df_user_items_test = pd.DataFrame(list(user_items_test.items()), columns=['customer_id', 'article_ids'])
df_user_items_test.tail()

,customer_id,article_ids
42508,6f75286145a0a6b97b94c957f4626147fbe0a3a83078e4...,"[854043005, 835639002]"
42509,4eadbb663d922efed1ebb6b354e967a26e30a6d4bc6131...,"[912204001, 806388009, 860720001, 730683050, 9..."
42510,a06da8807658e6627762b76e878ab242899c5545e07389...,"[839900001, 877222001, 877222001, 706016036, 7..."
42511,899a551b9798a97a55b49e21e00fd45175360c47acd250...,"[673677003, 661658004, 705734007, 575347014, 8..."
42512,3c0c198b6c04d50b639ca8128c3010603f34b37896bbf8...,"[768205001, 756040001, 731851001, 858678001, 7..."


In [12]:
df_transaction_train.head(5)

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id
0,0,2018-11-28,1f293f035403ce2d1ff0cdb23cc3faf8b1c2ad04dbedbe...,655265001,0.025407,2
1,1,2018-11-28,1f293f035403ce2d1ff0cdb23cc3faf8b1c2ad04dbedbe...,537688014,0.050831,2
2,2,2019-03-28,1f293f035403ce2d1ff0cdb23cc3faf8b1c2ad04dbedbe...,698877002,0.042356,2
3,3,2019-03-28,1f293f035403ce2d1ff0cdb23cc3faf8b1c2ad04dbedbe...,382127027,0.032186,2
4,4,2019-03-28,1f293f035403ce2d1ff0cdb23cc3faf8b1c2ad04dbedbe...,610776001,0.008458,2


In [13]:
from scipy import sparse
import numpy as np

user_items = {}
itemset = set()

# Construir user_items y itemset
for row in df_transaction_train.itertuples():
    if row[3] not in user_items:
        user_items[row[3]] = []
    user_items[row[3]].append(row[4])
    itemset.add(row[4])

itemset = np.sort(list(itemset))

# Inicializar listas para almacenar filas, columnas y valores de la matriz
rows = []
cols = []
vals = []

# Llenar las listas correspondientes con los valores de la matriz
for i, items in enumerate(user_items.values()):
    for item in items:
        rows.append(itemset.searchsorted(item))
        cols.append(i)
        vals.append(1)

# Crear la matriz dispersa CSR
user_item_matrix = sparse.csr_matrix((vals, (rows, cols)), shape=(len(itemset), len(user_items)))

user_ids = {key: i for i, key in enumerate(user_items.keys())}
# Transponer la matriz para tener usuarios como filas y artículos como columnas
user_item_matrix = user_item_matrix.T.tocsr()


In [14]:
user_item_matrix

<45000x73080 sparse matrix of type '<class 'numpy.int64'>'
	with 1125920 stored elements in Compressed Sparse Row format>

In [15]:
def evaluate_model(model, n):
  mean_map = 0.
  mean_ndcg = 0.
  for u in user_items_test.keys():
    user_index = user_ids[u]
    rec = model.recommend(user_index, user_item_matrix[user_index], n)[0]
    indices = []
    for item in user_items_test[u]:
        indices.append(list(itemset).index(item))
    rel_vector = [np.isin(user_items_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, n)

  mean_map /= len(user_items_test)
  mean_ndcg /= len(user_items_test)

  return mean_map, mean_ndcg

In [16]:
df_transaction_val.head(5)

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id
0,0,2020-09-20,1f293f035403ce2d1ff0cdb23cc3faf8b1c2ad04dbedbe...,535035001,0.118627,2
1,1,2020-06-15,b4ba419fd5afc5eb274ab3ae1638a51a2e916b4deb0a5c...,654046004,0.022017,2
2,2,2019-10-20,1a1563c28f0d8781ad87f76e0a5af0485003e350f541dd...,742086008,0.033881,2
3,3,2020-07-14,3219f007e898957cbe0ff0f22010cd7118c06f7498a167...,778534006,0.042356,1
4,4,2020-09-07,4e5c03f7ffefe2f99f8e0c9d9805c1571867e6a8dd7091...,706016015,0.033881,1


In [17]:
def show_recommendations(model, user, n):
  recommendations = model.recommend(userid=user, user_items=user_item_matrix[user], N=n)[0]
  return df_articles.loc[recommendations]['prod_name']

### Factorización matricial optimizada por ALS

In [18]:
model_als = implicit.als.AlternatingLeastSquares(factors=100, iterations=10, use_gpu=False)
model_als.fit(user_item_matrix)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

In [19]:
show_recommendations(model_als, user=77, n=10)

31918         Napolitana Dress
38165                Mezzo 3/4
14905      Straight col. denim
32810            Josefin Dress
11150          STINA 5-p boxer
27968        Evelina L/S Dress
51444           Blake Slim Trs
17611                 Stork SS
8507     Pat ls spring flanell
8924      Majorca cheeky breif
Name: prod_name, dtype: object

### Factorización matricial optimizada por BPR

In [20]:
model_bpr = implicit.bpr.BayesianPersonalizedRanking(factors=100, iterations=10, use_gpu=False)
model_bpr.fit(user_item_matrix)

  0%|          | 0/10 [00:00<?, ?it/s]

In [21]:
show_recommendations(model_bpr, user=77, n=10)

14995               Skinny Biker Trash
64952         Coachella Push Melbourne
94       Austin basic leather hip belt
17695                     Noah hood TP
18850                        Twist tee
30239                    HAVANA tights
2590               6P Tanktopbody pack
53083                       BILLIE set
2446                  Spearmint shorts
58031                 3P LS BODY MIXED
Name: prod_name, dtype: object

In [22]:
factor_values = [50, 100, 150, 200, 250]
for i in factor_values:
    maprec, ndcg = evaluate_model(model_als, i)
    print(f'n = {i}\map: {maprec}\nndcg: {ndcg}')

n = 50\map: 0.06688382526502636
ndcg: 0.11250676263731094
n = 100\map: 0.06688382526502636
ndcg: 0.11250676263731094
n = 150\map: 0.06688382526502636
ndcg: 0.11250676263731094
n = 200\map: 0.06688382526502636
ndcg: 0.11250676263731094
n = 250\map: 0.06688382526502636
ndcg: 0.11250676263731094


In [ ]:
factor_values = [50, 100, 150, 200, 250]
for i in factor_values:
    maprec, ndcg = evaluate_model(model_bpr, i)
    print(f'n = {i}\map: {maprec}\nndcg: {ndcg}')

n = 50\map: 0.06688382526502636
ndcg: 0.11250676263731094
n = 100\map: 0.06688382526502636
ndcg: 0.11250676263731094
